In [3]:
from utils.chat import OllamaChat
from utils.prompts import get_persona
from company_about_summarizer import summarize_company_about
from utils.example_company.company_about import company_data

# Summarizing the Company Data

In [13]:
company_data_summarized=summarize_company_about(company_data)


In [ ]:
system_prompt = get_persona("GENERAL_SALES_AGENT",company_data=company_data_summarized)

In [ ]:
print(system_prompt)

In [9]:
chat = OllamaChat(
        model='llama3.1',  # Replace with your desired model
        system_prompt=system_prompt,
        n_ctx=4096,
        temperature=0.7
    )

In [10]:
result=chat.chat(user_prompt="hello")
# chat._save_conversation()

In [12]:
print(result)

Hello! My name is Alex, and I'm a sales representative at TechCare AI. It's great to connect with you. How can I assist you today? Are you looking for ways to improve your customer service operations or optimize your existing systems? We've had tremendous success with our AI-powered solutions, and I'd love to explore how they might benefit your organization.


In [27]:
chat._load_conversation()

In [42]:
result=chatbot.chat(message="What Services Do you provide ?")